<a href="https://colab.research.google.com/github/will-gun/stock-data/blob/master/collectTaiwanStockData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
from io import StringIO
import requests
import pandas as pd
import numpy as np
import pickle

def create_today_timestamp():
    today = time.strftime("%Y-%m-%d",time.gmtime())
    return int(time.mktime(time.strptime(today, "%Y-%m-%d")))

def create_timestamp_from_today(n):
    today = create_today_timestamp()
    return today + n*24*3600
tomorrow_timestamp = create_timestamp_from_today(1)
print(tomorrow_timestamp)

def create_tw_stock_info_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    print(df)
    df.columns = df.iloc[0]
    df = df.iloc[2:900]
    df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
    #df = df.dropna(how='any')
    df = df.reset_index(drop=True)
    print(df)
    new_df = df['有價證券代號及名稱'].str.replace(u'\u3000',' ').str.split(u' ',expand=True)
    print(new_df)
    new_df.columns = ['Ticker', 'StockName']
    new_df['Sector'] = df['產業別']
    return new_df

tw_stock_info_df = create_tw_stock_info_list()
tw_stock_info_df

min = np.array([[100000.0,0],[100000.0,0],[100000.0,0],[100000.0,0],[100000.0,0],[100000.0,0]])
day = 750
day_1 = day-1

stock_df = pd.DataFrame()
ticker_list = tw_stock_info_df['Ticker']
#ticker_list = tw_stock_info_df['Ticker'].tail(5)
for ticker in ticker_list:
    print('## Info: Download Ticker '+ticker+'!')
    site = "https://query1.finance.yahoo.com/v7/finance/download/"+ticker+".TW?period1=0&period2="+str(tomorrow_timestamp)+"&interval=1d&events=history&crumb=hP2rOschxO0"
    try:
        response = requests.get(site)
        tmp_df = pd.read_csv(StringIO(response.text))
        tmp_df['Ticker'] = ticker
        #stock_df = pd.concat([stock_df,tmp_df],axis=0)
                  
    except:
        print('## Warning: Ticker '+ticker+' is failed!')

    tmp_df = tmp_df.reset_index(drop=True)
    if len(tmp_df.index) > day:
        tmp_df = tmp_df[len(tmp_df.index)-day:]
        tmp_df = tmp_df.reset_index(drop=True)
        print(tmp_df)
        abc = (tmp_df.at[day_1,'Adj Close'] - tmp_df.at[0,'Adj Close'])/day_1
        min[5,0]=0
        min[5,1]=ticker
        for i in range(day):
           min[5,0] += abs(tmp_df.at[i,'Adj Close']-abc*i+tmp_df.at[0,'Adj Close'])/(abc*i+tmp_df.at[0,'Adj Close'])
        min=min[min[:,0].argsort()]
        print(min)

#stock_df = stock_df.reset_index(drop=True)
#stock_df = stock_df[['Date','Ticker','Open','High','Low','Close','Adj Close','Volume']]
#print(stock_df)


Streaming output truncated to the last 5000 lines.
[[ 278.24440899 3661.        ]
 [ 305.05526611 1435.        ]
 [ 308.23777559 2368.        ]
 [ 311.52297146 3037.        ]
 [ 338.72376256 2303.        ]
 [1183.09359114 4108.        ]]
## Info: Download Ticker 4119!
           Date       Open       High  ...  Adj Close     Volume  Ticker
0    2018-01-18  61.400002  61.700001  ...  54.325516   166602.0    4119
1    2018-01-19  61.000000  61.400002  ...  53.969280   126156.0    4119
2    2018-01-22  60.599998  61.200001  ...  53.880222   151542.0    4119
3    2018-01-23  60.500000  60.500000  ...  53.256813   230877.0    4119
4    2018-01-24  59.500000  59.799999  ...  52.722466   225143.0    4119
..          ...        ...        ...  ...        ...        ...     ...
745  2021-02-18  91.800003  92.000000  ...  92.000000   780889.0    4119
746  2021-02-19  91.400002  94.000000  ...  93.599998   628176.0    4119
747  2021-02-22  94.500000  96.599998  ...  96.000000  1260080.0    4119
7